In [30]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle

# Load data
df = pd.read_csv('data/investigation_train_large_checked.csv')

# Create age groups
age_column = 'persoon_leeftijd_bij_onderzoek'
df['age_group'] = pd.qcut(df[age_column], q=4, labels=['youngest', 'young', 'middle', 'oldest'])

# Ensure checked column is boolean
df['checked'] = df['checked'].astype(bool)

# Create biased labels
young_mask = (df['age_group'] == 'youngest') | (df['age_group'] == 'young')
old_mask = (df['age_group'] == 'middle') | (df['age_group'] == 'oldest')

np.random.seed(42)
df.loc[young_mask & (np.random.random(len(df)) < 0.8), 'checked'] = True
df.loc[old_mask & (np.random.random(len(df)) < 0.3), 'checked'] = False

# Prepare features
features = [col for col in df.columns if col not in ['Ja', 'Nee', 'checked', 'age_group']]
X = df[features]
y = df['checked']

# Split and train
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42,
    stratify=df['age_group']
)

rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=10,
    random_state=42
)
rf.fit(X_train, y_train)

# Save
with open('models/bad_model.pkl', 'wb') as f:
    pickle.dump(rf, f)

test_data = {
    'X_test': X_test,
    'y_test': y_test,
    'features': features,
    'protected_indices': {
        'age_index': X_test.columns.get_loc(age_column)
    }
}
with open('data/test_data.pkl', 'wb') as f:
    pickle.dump(test_data, f)

In [11]:
X_test.columns.get_loc('persoon_leeftijd_bij_onderzoek')

216